In [584]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
#import tensorflow as tf 
pd.set_option('display.max_columns', 500)


# Preprocess Each Dataset (Fighters and Bouts)

In [585]:
#just fighters
fighters_data = pd.read_csv('Fights_scraper/spiders/scraped_fighters.csv') 
#just bouts
bouts_data = pd.read_csv('Bouts_Scraper/bouts_scraped/bouts_scraped/spiders/scraped_bouts.csv') 
#combined dataset, created with SQL Query, joined on fighter1 and 2 names
fighter_bouts = pd.read_csv('fighters_bouts_joined.csv') 


In [586]:
fighter_bouts = fighter_bouts.drop(columns=['round','time','win_method_finish','win_method_type'])

In [587]:
#shuffles winners in the df to make the classes of winners and losers balanced
import math
negative_index = np.random.choice(len(fighter_bouts),
                                  size= math.ceil(len(fighter_bouts)/2),
                                  replace = False)

fighter_bouts.iloc[negative_index,[2,3]] = fighter_bouts.iloc[negative_index,[3,2]].values



In [588]:
#make winner column align correctly with negative index
fighter_bouts['winner'].iloc[negative_index] = fighter_bouts['fighter2'].iloc[negative_index]

/Users/charlespierse/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [589]:
#creates two lists of the relevant columns stats for f1 and f2
f1_list = [col for col in fighter_bouts.columns if 'f1' in col]
f2_list = [col for col in fighter_bouts.columns if 'f2' in col]


In [590]:
f1_index = []
for col in fighter_bouts.columns:
    if col in f1_list:
        f1_index.append(fighter_bouts.columns.get_loc(col))

f2_index = []
for col in fighter_bouts.columns:
    if col in f2_list:
        f2_index.append(fighter_bouts.columns.get_loc(col))

In [591]:
fb_copy = fighter_bouts.copy()
#all values at the negative index of f1's become the values of negative index of f2's
fighter_bouts.iloc[negative_index,f1_index] = fighter_bouts.iloc[negative_index,f2_index].values
#all values at the negative index of f2's become the values of negative index of f1's using copy of fb_copy of fighter bouts
fighter_bouts.iloc[negative_index,f2_index] =fb_copy.iloc[negative_index,f1_index].values

In [592]:
#seperate df consisting only of categorical dtypes
categorical_fb = fighter_bouts.select_dtypes(include='object')
#drop those types for now from fighter bouts leaving only numeric values
fighter_bouts = fighter_bouts.drop(columns=categorical_fb.columns)


In [593]:
#some fighters have '--' as a null for their DOB and height
categorical_fb['f1_dob'] =  categorical_fb['f1_dob'].replace('--',None) 
categorical_fb['f2_dob'] =  categorical_fb['f2_dob'].replace('--',None)
#same as above for height
categorical_fb['f1_height'] =  categorical_fb['f1_height'].replace('--',None) 
categorical_fb['f2_height'] =  categorical_fb['f2_height'].replace('--',None)

In [594]:
categorical_fb['event_date'] = pd.to_datetime(categorical_fb['event_date'])
categorical_fb['f1_dob'] = pd.to_datetime(categorical_fb['f1_dob'])
categorical_fb['f2_dob'] = pd.to_datetime(categorical_fb['f2_dob'])


In [595]:
#weights can be seen as an ordinal category value so this list declares that order
weights_ordered = ["Women's Strawweight","Women's Flyweight","Women's Bantamweight",
               "Women's Featherweight", "Flyweight", "Bantamweight", "Featherweight",
               "Lightweight","Welterweight", "Middleweight","Light Heavyweight",
               "Heavyweight", "Super Heavyweight", "Open Weight","Catch Weight"]



In [596]:
categorical_fb["weight_class"] = categorical_fb.weight_class.astype("category",
                                 ordered=True,
                                 categories= weights_ordered).cat.codes


/Users/charlespierse/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [597]:
#Creating a new feature for fighter1 and fighter2: ageAtFight. It's ordinal so no need to one hot encode,
categorical_fb['f1_ageAtFight'] = (categorical_fb.event_date - categorical_fb.f1_dob).dt.days
categorical_fb['f2_ageAtFight'] = (categorical_fb.event_date - categorical_fb.f2_dob).dt.days

In [598]:
#starting to clean up the the fighters' records categories, first we replace text ('Record') with nothing
categorical_fb = categorical_fb.replace('Record: ',"",regex=True) 

In [599]:
#make these values all zero (just in case something goes wrong with record_split()
categorical_fb['f1_win'],categorical_fb['f1_loss'],categorical_fb['f1_draw'],categorical_fb['f1_nc'] = 0,0,0,0
categorical_fb['f2_win'],categorical_fb['f2_loss'],categorical_fb['f2_draw'],categorical_fb['f2_nc'] = 0,0,0,0


In [600]:
#a small function to handle the remainder of the record strings
#takes in a row split the values on '-'
#if 'NC' is contained the draw var(the only place it could be) then we split draw on brackets first
#draw is simply equal to the 0th element of that split
#nc is equal to the 1st element of that split with a regex to remove any remaining non numeric values
#cast all values to int and return all 4 
import re

def record_split(row):
        win,loss, draw = row.split('-')
        nc= 0
        if 'NC' in draw:       
            draw = draw.split('(')
            nc = re.sub('[^0-9]','', draw[1])
            draw = draw[0]
        else:
            nc = 0     
            
        win,loss, draw, nc = int(win),int(loss), int(draw), int(nc)
            
        return win,loss,draw,nc
        

In [601]:
#this is a messy way to do these assignments but works. 
#use the pd apply function to apply our record_split() function on each row in df
#zip(*...) unpacks our results nicely
categorical_fb['f1_win'],\
categorical_fb['f1_loss'],\
categorical_fb['f1_draw'],\
categorical_fb['f1_nc'] = zip(*categorical_fb.apply(lambda x: record_split(x['f1_record']), axis=1))

categorical_fb['f2_win'],\
categorical_fb['f2_loss'],\
categorical_fb['f2_draw'],\
categorical_fb['f2_nc'] = zip(*categorical_fb.apply(lambda x: record_split(x['f2_record']), axis=1))



In [602]:
categorical_fb = categorical_fb.drop(columns =['f1_record','f2_record'])

In [603]:

#adds winner to end of column for readability 
cols = list(categorical_fb.columns.values) 
cols.pop(cols.index('winner')) 
categorical_fb = categorical_fb[cols+['winner']] 



In [604]:
categorical_fb.head()

,event_date,figher1,fighter2,weight_class,f1_dob,f1_height,f1_stance,f2_dob,f2_height,f2_stance,f1_ageAtFight,f2_ageAtFight,f1_win,f1_loss,f1_draw,f1_nc,f2_win,f2_loss,f2_draw,f2_nc,winner
0,2018-01-14,Jeremy Stephens,Dooho Choi,6,1986-05-25,5' 8,Orthodox,1991-04-10,5' 10,Orthodox,11557,9776,28,16,0,0,14,3,0,0,Jeremy Stephens
1,2018-01-14,Jessica-Rose Clark,Paige VanZant,1,1987-11-28,5' 5,Orthodox,1994-03-26,5' 4,Orthodox,11005,8695,9,5,0,1,8,4,0,0,Jessica-Rose Clark
2,2018-01-14,Emil Meek,Kamaru Usman,8,1988-08-20,5' 11,Switch,1987-05-11,6' 0,Switch,10739,11206,9,4,1,1,15,1,0,0,Kamaru Usman
3,2018-01-14,Darren Elkins,Michael Johnson,6,1984-05-16,5' 10,Orthodox,1986-06-04,5' 10,Southpaw,12296,11547,25,7,0,0,20,14,0,0,Darren Elkins
4,2018-01-14,Alex White,James Krause,7,1988-10-22,6' 0,Southpaw,1986-06-04,6' 2,Orthodox,10676,11547,13,5,0,0,25,8,0,0,James Krause


In [605]:
def parse_height(height):
    #expected format is 5' 10, 6'3 etc
    ht = height.split("' ")
    ft = float(ht[0])
    inch = float(ht[1])
    return (12*ft) + inch
        

In [606]:
categorical_fb.f2_height = categorical_fb.f2_height.apply(lambda x: parse_height(x))
categorical_fb.f1_height = categorical_fb.f1_height.apply(lambda x: parse_height(x))

In [607]:
# categorical_fb = categorical_fb.join(pd.get_dummies(categorical_fb.win_method_type,prefix= 'win_method'))
# categorical_fb = categorical_fb.join(pd.get_dummies(categorical_fb.win_method_finish,prefix= 'win_finish'))
categorical_fb = categorical_fb.join(pd.get_dummies(categorical_fb.f1_stance,prefix= 'f1_stance'))
categorical_fb = categorical_fb.join(pd.get_dummies(categorical_fb.f2_stance,prefix= 'f2_stance'))

In [608]:
categorical_fb = categorical_fb.drop(columns=['f1_stance','f2_stance','f1_dob','f2_dob','event_date'])

In [609]:

#adds winner to end of column for readability 
cols = list(categorical_fb.columns.values) 
cols.pop(cols.index('winner')) 
categorical_fb = categorical_fb[cols+['winner']] 



In [610]:
categorical_fb['winner'] = (categorical_fb['fighter2'] == categorical_fb['winner']).astype('int')
categorical_fb['figher1'] = 0
categorical_fb['fighter2'] = 1

In [611]:
fbs_joined = pd.concat([fighter_bouts,categorical_fb],axis=1)

In [612]:
target = fbs_joined.winner

In [613]:
fbs_joined = fbs_joined.drop(columns=['event_attendence'])

In [614]:
fbs_joined = fbs_joined.drop(columns=['winner'])

In [615]:
#we'll be using this later for the second model prep
fbj_copy = fbs_joined.copy()

In [616]:
fbs_joined.shape

(4851, 53)

In [617]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy= 'median',copy= False)
scaler = StandardScaler()

In [618]:
scaled_fbs = scaler.fit_transform(fbs_joined)
imputed_fbs = imputer.fit_transform(scaled_fbs)

/Users/charlespierse/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/charlespierse/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [619]:
from sklearn.model_selection import StratifiedShuffleSplit
sss= StratifiedShuffleSplit(n_splits=20,test_size=0.2,random_state=42)

for train_index, test_index in sss.split(imputed_fbs, target):
    X_train, X_test = imputed_fbs[train_index], imputed_fbs[test_index]
    y_train, y_test = target[train_index], target[test_index]

In [620]:
np.savetxt("model_data/X_train.csv", X_train, delimiter=",")
np.savetxt("model_data/y_train.csv", y_train, delimiter=",")
np.savetxt("model_data/X_test.csv", X_test, delimiter=",")
np.savetxt("model_data/y_test.csv", y_test, delimiter=",")

# Fight Stats Prediction Data Prep

This next step is about preparing the data for the second model that will be used.
The aim of this model is to produce multiple linear regressions that will try to predict the 
performance (fight stats) of two fighters in a bout and then pass those on to our main Neural Net

In [621]:
fbj_copy.shape

(4851, 53)

In [622]:
predictor_cols = ['pass_stat_f1', 'pass_stat_f2', 'str_stat_f1', 'str_stat_f2',
       'sub_stat_f1', 'sub_stat_f2', 'td_stat_f1', 'td_stat_f2',]

In [623]:
fbj_copy[predictor_cols].head()

,pass_stat_f1,pass_stat_f2,str_stat_f1,str_stat_f2,sub_stat_f1,sub_stat_f2,td_stat_f1,td_stat_f2
0,0,0,49,44,0,0,0,0
1,4,0,55,54,1,0,2,0
2,0,7,30,50,2,0,0,8
3,2,0,18,45,1,0,1,0
4,0,4,56,35,0,2,1,3


In [624]:
targets = fbj_copy[predictor_cols]


In [625]:
fbj_copy = fbj_copy.drop(columns= targets.columns)

In [626]:
fbj_copy.shape

(4851, 45)

In [627]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer
imputer = SimpleImputer(strategy= 'median',copy= False)
scaler = StandardScaler()
normalizer = Normalizer()

In [628]:
scaled_fbj = scaler.fit_transform(fbj_copy)
imputed_fbj = imputer.fit_transform(fbj_copy)

/Users/charlespierse/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/charlespierse/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [629]:
from sklearn.model_selection import train_test_split
X_train_stat, X_test_stat, y_train_stat, y_test_stat = train_test_split(imputed_fbj, targets.values, test_size=0.10, random_state=42)


In [630]:
np.savetxt("model_data/X_train_stats.csv", X_train_stat, delimiter=",")
np.savetxt("model_data/y_train_stats.csv", y_train_stat, delimiter=",")
np.savetxt("model_data/X_test_stats.csv", X_test_stat, delimiter=",")
np.savetxt("model_data/y_test_stats.csv", y_test_stat, delimiter=",")

# Loading In Fights Stats Predictor Model

In [631]:
import tensorflow.keras.backend as K

def r2(y_true, y_pred):
    SS_res = K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true)))
    return 1 - SS_res/(SS_tot + K.epsilon())

In [632]:
import tensorflow as tf

model = tf.keras.models.load_model('Saved_Models/Fight_Stats_Models/stats_model2019-05-25.h5',custom_objects={"r2": r2})

# Using the loaded model to predict two fighters' stats in a bout

In [577]:
def predict_bout_stats(fighters):
    stats_predicted = model.predict(fighters)
    return stats_predicted

In [578]:
fbj_copy.head()


,pass_stat_f1,pass_stat_f2,str_stat_f1,str_stat_f2,sub_stat_f1,sub_stat_f2,td_stat_f1,td_stat_f2,f1_reach,f1_sapm,f1_slpm,f1_stk_acc,f1_stk_def,f1_sub_avg,f1_td_acc,f1_td_avg,f1_td_def,f1_weight,f2_reach,f2_sapm,f2_slpm,f2_stk_acc,f2_stk_def,f2_sub_avg,f2_td_acc,f2_td_avg,f2_td_def,f2_weight,figher1,fighter2,weight_class,f1_height,f2_height,f1_ageAtFight,f2_ageAtFight,f1_win,f1_loss,f1_draw,f1_nc,f2_win,f2_loss,f2_draw,f2_nc,f1_stance_Open Stance,f1_stance_Orthodox,f1_stance_Sideways,f1_stance_Southpaw,f1_stance_Switch,f2_stance_Open Stance,f2_stance_Orthodox,f2_stance_Sideways,f2_stance_Southpaw,f2_stance_Switch
0,0,0,49,44,0,0,0,0,71.0,2.84,3.08,40,59,0.4,36,1.11,64,145.0,70.0,6.26,5.64,53,54,0.0,20,0.55,50,145.0,0,1,6,68.0,70.0,11557,9776,28,16,0,0,14,3,0,0,0,1,0,0,0,0,1,0,0,0
1,4,0,55,54,1,0,2,0,64.0,4.13,4.60,50,61,0.7,66,1.33,57,125.0,65.0,2.63,3.40,52,43,1.0,33,1.31,35,115.0,0,1,1,65.0,64.0,11005,8695,9,5,0,1,8,4,0,0,0,1,0,0,0,0,1,0,0,0
2,7,0,50,30,0,2,8,0,76.0,1.60,4.20,54,57,0.2,50,3.96,100,170.0,74.0,2.64,2.24,38,32,1.3,50,0.33,48,170.0,0,1,8,72.0,71.0,11206,10739,15,1,0,0,9,4,1,1,0,0,0,0,1,0,0,0,0,1
3,2,0,18,45,1,0,1,0,71.0,2.83,3.36,37,53,1.3,35,2.68,57,145.0,73.0,3.82,4.15,37,58,0.1,45,0.56,78,145.0,0,1,6,70.0,70.0,12296,11547,25,7,0,0,20,14,0,0,0,1,0,0,0,0,0,0,1,0
4,0,4,56,35,0,2,1,3,71.0,2.87,3.77,42,62,0.6,50,1.05,72,155.0,73.0,4.15,4.57,46,61,0.7,18,1.18,46,170.0,0,1,7,72.0,74.0,10676,11547,13,5,0,0,25,8,0,0,0,0,0,1,0,0,1,0,0,0


In [635]:
prediction = predict_bout_stats(X_train_stat)

In [641]:
print(prediction[110])

[ 2.062191    0.40079272 25.567804   21.898308    0.937916    0.704941
  1.742553    0.25067925]


In [640]:
print(y_train_stat[110])

[ 4  0 12  8  0  1  2  0]


In [535]:
fighters_data.head()

,date_of_birth,fighter_name,fighter_record,height,reach,sapm,slpm,stance,strike_acc,strike_def,sub_avg,td_acc,td_avg,td_def,weight
0,Jul 17 1990,Michael Byrnes,Record: 5-2-0,"5' 11""",NaN,0.00,0.00,NaN,0,0.0,0.0,0,0.00,0,155.0
1,Feb 07 1989,Gleidson Cutis,Record: 7-3-0,"5' 9""",NaN,8.28,2.99,Orthodox,52,59.0,0.0,0,0.00,0,155.0
2,Jun 25 1981,Luciano Azevedo,Record: 16-9-1,"6' 3""",NaN,1.97,0.76,Orthodox,45,27.0,0.0,11,2.28,0,161.0
3,Nov 10 1980,Katsuaki Furuki,Record: 1-1-0,"6' 0""",NaN,2.67,0.27,NaN,10,39.0,3.0,0,0.00,0,168.0
4,Aug 11 1990,Rolando Dy,Record: 9-7-1 (1 NC),"5' 8""",69.0,4.47,3.04,Orthodox,37,52.0,0.0,20,0.30,68,145.0
